# Wikipedia Notable Life Expectancies

# [Notebook 5 of : Data Cleaning](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean4_thanak_2022_06_23.ipynb)

## Context

The


## Objective

The

### Data Dictionary

- Feature: Description

## Importing Necessary Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To save/open python objects in pickle file
import pickle

# To help with reading, cleaning, and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

# To play auditory cue when cell has executed, has warning, or has error and set chime theme
import chime

chime.theme("zelda")

<IPython.core.display.Javascript object>

## Data Overview

### Reading, Sampling, and Checking Data Shape

In [2]:
# Reading the dataset
conn = sql.connect("wp_life_expect_clean3.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_clean3", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 132652 rows and 23 columns.


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January,None,None,dancer,ballet designer and director,None,None,None,None,None,None,None,None,86.0,None,United Kingdom of Great Britain and Northern Ireland,None
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January,None,None,economist,writer,and academic,None,None,None,None,None,None,None,68.0,None,Ireland,None


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df.tail(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
132650,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2,2022,June,1980,None,volleyball player,Olympic champion and coach,None,None,None,None,None,None,None,None,69.0,None,Russia,None
132651,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3,2022,June,None,None,engineer,member of the Chinese Academy of Engineering,None,None,None,None,None,None,None,None,86.0,None,"China, People's Republic of",None


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df.sample(5)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
118382,15,Anatoliy Vishnevskiy,", 85, Russian economist and demographer.",https://en.wikipedia.org/wiki/Anatoliy_Vishnevskiy,5,2021,January,None,None,economist and demographer,None,None,None,None,None,None,None,None,None,85.0,None,Russia,None
41080,12,S. G. Sender,", 78, Belgian pastry chef, after long illness.",https://en.wikipedia.org/wiki/S._G._Sender,1,2009,July,None,None,pastry chef,after long illness,None,None,None,None,None,None,None,None,78.0,None,Belgium,None
101218,9,Nicholas Kafoglis,", 89, American politician, member of the Kentucky House of Representatives , complications from broken hip.",https://en.wikipedia.org/wiki/Nicholas_Kafoglis,4,2019,February,1972 1976 and Senate 1988 1998,None,politician,member of the Kentucky House of Representatives,complications from broken hip,None,None,None,None,None,None,None,89.0,None,United States of America,None
44552,13,Luis Antonio Chávez,", 22, Honduran journalist and children's radio host, shot.",https://en.wikipedia.org/wiki/Luis_Antonio_Ch%C3%A1vez,1,2010,April,None,None,journalist and children's radio host,shot,None,None,None,None,None,None,None,None,22.0,None,Honduras,None
74718,5,Richard Dysart,", 86, American actor , cancer.",https://en.wikipedia.org/wiki/Richard_Dysart,6,2015,April,"Emmy, 1992, , winner",None,actor,cancer,None,None,None,None,None,None,None,None,86.0,None,United States of America,None


<IPython.core.display.Javascript object>

### Checking Data Types, Duplicates, and Null Values

In [5]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132652 entries, 0 to 132651
Data columns (total 23 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   day             132652 non-null  object 
 1   name            132652 non-null  object 
 2   info            132652 non-null  object 
 3   link            132652 non-null  object 
 4   num_references  132652 non-null  object 
 5   year            132652 non-null  int64  
 6   month           132652 non-null  object 
 7   info_parenth    49830 non-null   object 
 8   info_1          35 non-null      object 
 9   info_2          132604 non-null  object 
 10  info_3          62571 non-null   object 
 11  info_4          12605 non-null   object 
 12  info_5          1497 non-null    object 
 13  info_6          216 non-null     object 
 14  info_7          31 non-null      object 
 15  info_8          6 non-null       object 
 16  info_9          1 non-null       object 
 17  info_10   

<IPython.core.display.Javascript object>

#### Loading `nation_map` from Pickle File to Dictionary nation_map

In [6]:
# Load the nation_map
with open("nation_map.pkl", "rb") as f:
    nation_map = pickle.load(f)

<IPython.core.display.Javascript object>

#### Loading `other_species` list from other_species.csv

In [7]:
# Loading other_species list
other_species_df = pd.read_csv("other_species.csv")
other_species = other_species_df["species"].tolist()

<IPython.core.display.Javascript object>

#### Observations:
- With our dataframe, `nation_map`, and `other_species` list loaded, we can proceed to extracting the other features.
- First, we will clean up the divided `info` columns by removing any remaining digits and nationality and country values.
- We will use the same functions from previous notebooks.

#### Function to Save Indices of Rows Matching Regular Expressions Pattern to a List and Print Number of Rows with Match 

In [8]:
# Define a function that takes dataframe, column name, and re pattern as arguments and returns list of indices
# for which column value matches re pattern
def rows_with_pattern(dataframe, column, pattern):
    """
    Takes input of dataframe, column name, and re pattern 
    and returns list of indices for rows that contain match
    for pattern anywhere within value for given column.
    
    dataframe: dataframe
    column: column name
    pattern: re pattern
    """
    index_list = []

    for i in dataframe.index:
        item = dataframe.loc[i, column]
        match = re.search(pattern, item)
        if match:
            index_list.append(i)
    print(
        f"There are {len(index_list)} rows with matching pattern in column '{column}'."
    )
    return index_list

<IPython.core.display.Javascript object>

#### Function to Use rows_with_pattern Function for Multiple Regular Expression Patterns

In [9]:
# Define a function that calls rows_with_pattern function for multiple re patterns
# returning a single list of indices for all rows with any pattern match


def multiple_patterns(dataframe, column, patterns):
    """
    Takes input dataframe, column, and list of re patterns and returns single list 
    of indices for rows in which a match for any pattern is found with re.search
    
    dataframe: dataframe
    column: column name
    patterns: list of re patterns
    """
    rows_combined = []

    # For loop to check each pattern
    for pattern in patterns:

        # List and number of rows matching each pattern
        print(pattern)
        rows_to_check = rows_with_pattern(dataframe, column, pattern)
        print("")

        # Add list for each pattern to combined list
        rows_combined += rows_to_check

    return rows_combined

<IPython.core.display.Javascript object>

### Removing Remaining Digits and Nationality/Country Values from Divided `info` Columns

#### List of Columns to Treat

In [10]:
# List of columns to treat
cols_lst = [
    "info_1",
    "info_2",
    "info_3",
    "info_4",
    "info_5",
    "info_6",
    "info_7",
    "info_8",
    "info_9",
    "info_10",
    "info_11",
    "info_parenth",
]

<IPython.core.display.Javascript object>

#### Removing Digits

In [11]:
# Regular expression for parenthesis and its contents
pattern = r"\d"

# For loop to find indices of rows that have pattern
rows_combined = []
for column in cols_lst:
    dataframe = df[df[column].notna()]
    rows_to_check = rows_with_pattern(dataframe, column, pattern)
    rows_combined += rows_to_check

# Checking a sample of rows
df.loc[rows_combined, :].sample(2)

There are 0 rows with matching pattern in column 'info_1'.
There are 442 rows with matching pattern in column 'info_2'.
There are 2252 rows with matching pattern in column 'info_3'.
There are 1060 rows with matching pattern in column 'info_4'.
There are 69 rows with matching pattern in column 'info_5'.
There are 5 rows with matching pattern in column 'info_6'.
There are 0 rows with matching pattern in column 'info_7'.
There are 0 rows with matching pattern in column 'info_8'.
There are 0 rows with matching pattern in column 'info_9'.
There are 0 rows with matching pattern in column 'info_10'.
There are 0 rows with matching pattern in column 'info_11'.
There are 24403 rows with matching pattern in column 'info_parenth'.


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
107185,1,Noksong Boham,", 71, Indian politician, MLA , cancer.",https://en.wikipedia.org/wiki/Noksong_Boham,7,2019,December,"1978 1980, 1984 1999",None,politician,MLA,cancer,None,None,None,None,None,None,None,71.0,None,India,None
54752,30,Sicelo Shiceka,", 45, South African politician, Minister of Cooperative Governance and Traditional Affairs .",https://en.wikipedia.org/wiki/Sicelo_Shiceka,11,2012,April,since 2009,None,politician,Minister of Cooperative Governance and Traditional Affairs,None,None,None,None,None,None,None,None,45.0,None,South Africa,None


<IPython.core.display.Javascript object>

In [12]:
# For loop to extract digits
for column in cols_lst:
    for index in set(rows_combined):
        item = df.loc[index, column]
        if item:
            match = re.search(pattern, item)
            if match:
                df.loc[index, column] = re.sub(pattern, "", item)

# Rechecking number and example rows after treatment
# For loop to find indices of rows that have pattern
recheck_rows = []
for column in cols_lst:
    dataframe = df[df[column].notna()]
    rows_to_check = rows_with_pattern(dataframe, column, pattern)
    recheck_rows += rows_to_check

There are 0 rows with matching pattern in column 'info_1'.
There are 0 rows with matching pattern in column 'info_2'.
There are 0 rows with matching pattern in column 'info_3'.
There are 0 rows with matching pattern in column 'info_4'.
There are 0 rows with matching pattern in column 'info_5'.
There are 0 rows with matching pattern in column 'info_6'.
There are 0 rows with matching pattern in column 'info_7'.
There are 0 rows with matching pattern in column 'info_8'.
There are 0 rows with matching pattern in column 'info_9'.
There are 0 rows with matching pattern in column 'info_10'.
There are 0 rows with matching pattern in column 'info_11'.
There are 0 rows with matching pattern in column 'info_parenth'.


<IPython.core.display.Javascript object>

#### Removing Any Remaining Matches with  `nation_map` Keys and Values

In [13]:
%%time

# For loop to extract remaining information matching items in nation_map
for column in cols_lst:
    dataframe = df[df[column].notna()]
    for nationality, country in nation_map.items():
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if nationality in item or country in item:
                    df.loc[index, column] = item.replace(nationality, "").strip().replace(country,'').strip()

CPU times: total: 13min 10s
Wall time: 13min 10s


<IPython.core.display.Javascript object>

#### Observations:
- After that bit of tidying, we can proceed to extracting `known_for_1` values.
- The bulk of these values should be in `info_2`, according to the Wikipedia defined fields, so we will start there.

## Extracting `known_for` Data
Our goal will be to have some broader categories into which the specific values will fit.  `known_for` is a diverse feature, in that an individual may be known for a long-term role or roles, a specific event, a relationship with another person who is famous, etc.  So, to some extent we will see what we find and adapt as we go.

Also, we will abandon searching left to right as an individual may fit more than one category, and in no particular order.  For example, Ronald Reagan, is entered as "American actor and politician".  He is most known as the 40th president of The United States, so if we prioritized the first value, he would fit only into the category containing actor.  At the same time, it may have been his acting career that led to his political career.  Both arenas are relevant, so we will aim to capture all categories for an individual.  Later, when there are duplicate categories for an indivual, we can remove the redundant values.

We will take the following approach:
1. create and check a list of unique values in `info_2` that have a minimum number repeated, sufficient to create sets for each category, but not so exhaustive to be time prohibitive to manually enter.
2. using the pop() method, add each role to it's associated category's set, below.
3. combine the sets for each category into one dictionary.
4. search for the values in the dictionary and extract the category key value to a new column `known_for_1`, `known_for_2`, etc.

In [14]:
# Obtaining values for column and their counts
col_values = df["info_2"].value_counts()

# Creating a list for values that occur more than set number of time
roles_list = [index for index in col_values.index if col_values[index] > 30]

# Checking length of list
print(f"We will examine the top {len(roles_list)} unique values in info_2.")

We will examine the top 448 unique values in info_2.


<IPython.core.display.Javascript object>

In [15]:
# # Using pop to check list items and add to associated dictionary below
# roles_list.pop()

<IPython.core.display.Javascript object>

In [34]:
# Creating sets for each category
politics_govt_law = set(
    [
        "politician",
        "economist",
        "attorney",
        "trade unionist",
        "unionist",
        "aristocrat",
        "diplomat",
        "lawyer",
        "activist",
        "civil rights",
        "federal",
        "judge",
        "political",
        "politic",
        "royal",
        "civil servant",
        "jurist",
        "judge",
        "conservationist",
        "government official",
        "government",
        "barrister",
        "militant",
        "environmentalist",
        "public servant",
        "King",
        "Queen",
        "Princess",
        "Prince",
        "President",
        "Prime Minister",
        "leader",
    ]
)
arts = set(
    [
        "actor",
        "dancer",
        "choreographer",
        "model",
        "television",
        "jazz",
        "singer",
        "composer",
        "conductor",
        "journalist",
        "writer",
        "saxophonist",
        "film director",
        "comedian",
        "photojournalist",
        "poet",
        "actress",
        "film",
        "editor",
        "drummer",
        "producer",
        "songwriter",
        "publisher",
        "author",
        "violinist",
        "rapper",
        "musician",
        "animator",
        "filmmaker",
        "pianist",
        "historian",
        "comic",
        "screenwriter",
        "fashion",
        "designer",
        "guitarist",
        "voice",
        "opera",
        "cinematographer",
        "playwright",
        "sculptor",
        "novelist",
        "photographer",
        "architect",
        "painter",
        "artist",
        "disc jockey",
        "dj",
        "DJ",
        "MC",
        "bridge player",
        "tenor",
        "trombonist",
        "filmmaker",
        "ballerina",
        "bassist",
        "film critic",
        "critic",
        "personality",
        "organist",
        "operatic",
        "lyricist",
        "translator",
        "visual artist",
        "soprano",
        "cellist",
        "broadcaster",
        "chef",
        "literary critic",
        "ballet",
        "illustrator",
        "theatre director",
        "trumpeter",
        "presenter",
        "sportscaster",
        "cartoonist",
        "sportswriter",
        "choral",
    ]
)
sports = set(
    [
        "football",
        "footballer",
        "Olympic",
        "skier",
        "hockey",
        "soccer",
        "cricket",
        "soccer",
        "sprinter",
        "equestrian",
        "gymnast",
        "fencer",
        "chess",
        "wrestler",
        "swimmer",
        "basketball",
        "hurler",
        "sailor",
        "rower",
        "rugby",
        "athlete",
        "golfer",
        "boxer",
        "tennis",
        "cyclist",
        "racing",
        "driver",
        "cricketer",
        "baseball",
        "speedway rider",
        "speedway",
        "rider",
        "badminton",
        "sport shooter",
        "runner",
        "umpire",
        "judoka",
        "volleyball",
        "track and field",
        "track",
        "bobsledder",
        "canoer",
        "bodybuilder",
        "skater",
        "curler",
        "Olympic diver",
        "martial artist",
        "racer",
        "handball",
        "ski jumper",
        "racehorse trainer",
        "racecar driver",
        "hurdler",
        "polo",
        "Olympic shooter",
        "weightlifter",
        "Baseball",
        "mountaineer",
        "jockey",
        "Olympic sports shooter",
        "referee",
    ]
)
sciences = set(
    [
        "engineer",
        "physicist",
        "geologist",
        "psychiatrist",
        "botanist",
        "biologist",
        "anthropologist",
        "astronomer",
        "biochemist",
        "scientist",
        "computer",
        "archaeologist",
        "psychologist",
        "sociologist",
        "physician",
        "chemist",
        "physicist",
        "mathematician",
        "cosmonaut",
        "pediatrician",
        "astronaut",
        "entomologist",
        "cardiologist",
        "doctor",
        "nurse",
        "immunologist",
        "meteorologist",
        "medical researcher",
        "ornithologist",
        "neuroscientist",
        "microbiologist",
        "zoologist",
        "geographer",
        "inventor",
        "geneticist",
        "surgeon",
        "astrophysicist",
        "statistician",
    ]
)
business = set(
    [
        "executive",
        "business",
        "businessman",
        "banker",
        "entrepreneur",
        "real estate developer",
        "restaurateur",
        "businesswoman",
    ]
)
academia = set(
    [
        "scholar",
        "linguist",
        "educator",
        "philosopher",
        "academic",
        "historian",
        "educationalist",
        "philologist",
        "librarian",
        "industrialist",
        "professor",
        "musicologist",
    ]
)
law_enf_military_operator = set(
    [
        "officer",
        "army",
        "Army",
        "general",
        "police",
        "admiral",
        "soldier",
        "Air Force",
        "intelligence",
        "major",
        "lieutenant",
        "admiral",
        "fighter pilot",
        "pilot",
        "naval",
        "Navy",
        "aviator",
    ]
)
religion = set(
    [
        "rabbi",
        "Catholic",
        "priest",
        "Anglican",
        "cardinal",
        "theologian",
        "prelate",
        "Orthodox",
        "Episcopal",
        "bishop",
        "Jesuit",
        "hierarch",
        "Islamic",
        "religious leader",
    ]
)
social = set(["philanthropist", "socialite"])
crime = set(
    ["serial killer", "murderer", "convicted", "terrorist", "mobster", "criminal"]
)
victim = set(["Holocaust survivor", "victim"])
age = set(["supercentenarian", "oldest person"])
other_species.append("Tree")

<IPython.core.display.Javascript object>

#### Observations:
- We have a good start on `known_for_1` values for which to search.  Some other roles that have been observed previously we have added to the list also.
- Note that roles such as sportswriter and sports broadcaster, though associated with sports, are included in arts, to align with the underlying nature of the work itself.
- Let us combine them into one dictionary, taking care to put arts last to avoid missing values for "martial artist" and to put religion before politics_govt_law so that "leader" in politics_govt_law comes after "religious leader" in relgion.
- We will also include an other_species category here, again putting it last so that trainer and breeder in sports, come before racehorse in other_species.
- Then, we can proceed to extract the category to a new column, `known_for_1`.

#### Creating `known_for_dict` Dictionary of Category Keys and Specific Role Sets of Values

In [37]:
# Combining separate dictionaries into one dictionary putting arts last
known_for_dict = {
    "age": age,
    "victim": victim,
    "crime": crime,
    "social": social,
    "religion": religion,
    "law_enf_military_operator": law_enf_military_operator,
    "academia": academia,
    "business": business,
    "sciences": sciences,
    "sports": sports,
    "politics_govt_law": politics_govt_law,
    "arts": arts,
    "other_species": other_species,
}

<IPython.core.display.Javascript object>

#### Extracting Category to `known_for_1` Column from `info_1`

In [18]:
# Initializing known_for_1 column
df["known_for_1"] = ""

<IPython.core.display.Javascript object>

In [19]:
%%time

# Column to check
column = 'info_1'

# Extract to column
extract_to = 'known_for_1'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!=''].sample(2)

There are 29 values in extract_to column.
CPU times: total: 1.73 s
Wall time: 1.72 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1
8861,13,George Strugar,", 63. American gridiron football player, lung cancer.",https://en.wikipedia.org/wiki/George_Strugar,0,1997,June,None,gridiron player,lung cancer,None,None,None,None,None,None,None,None,None,63.0,None,United States of America,None,sports
32981,28,David Turnbull,". 92, American materials scientist.",https://en.wikipedia.org/wiki/David_Turnbull_(materials_scientist),8,2007,April,None,materials,None,None,None,None,None,None,None,None,None,None,92.0,None,United States of America,None,sciences


<IPython.core.display.Javascript object>

#### Observations:
- Once again, the `info_1` column has provided a small sample on which to test our code, which appears to be working.
- We can move on to extracting additional `known_for` values in `info_1` to `known_for_2`.
- Sir Robin Brook is a good example of an individual who would have 3 categories with our approach--business, business, and sports.  So, we will have enough `known_for` columns to extract all values for all entries.  Removing these values has the added benefit of simplifying the next search for `cause_of_death`.

#### Extracting Category to `known_for_2` Column from `info_1`

In [20]:
# Initializing known_for_2 column
df["known_for_2"] = ""

<IPython.core.display.Javascript object>

In [21]:
%%time

# Column to check
column = 'info_1'

# Extract to column
extract_to = 'known_for_2'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df['known_for_1']!= '') & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!=''].sample(2)

There are 8 values in extract_to column.
CPU times: total: 3.08 s
Wall time: 3.07 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1,known_for_2
99212,5,Kenneth Roy,", 73. Scottish broadcaster and writer.",https://en.wikipedia.org/wiki/Kenneth_Roy,2,2018,November,None,and,None,None,None,None,None,None,None,None,None,None,73.0,None,Scotland,None,arts,arts
66902,26,Bill Roetzheim,", 85. American Olympic gymnast.",https://en.wikipedia.org/wiki/Bill_Roetzheim,16,2014,February,None,,None,None,None,None,None,None,None,None,None,None,85.0,None,United States of America,None,sports,sports


<IPython.core.display.Javascript object>

#### Extracting Category to `known_for_3` Column from `info_1`

In [29]:
# Initializing known_for_2 column
df["known_for_3"] = ""

<IPython.core.display.Javascript object>

In [32]:
%%time

# Column to check
column = 'info_1'

# Extract to column
extract_to = 'known_for_3'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df['known_for_2']!= '') & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')

There are 0 values in extract_to column.
CPU times: total: 2.69 s
Wall time: 2.69 s


<IPython.core.display.Javascript object>

In [36]:
# Checking remaining unique values in info_1
df["info_1"].value_counts()

                    16
early                2
professional         1
choral               1
college  coach       1
player               1
and                  1
common               1
materials            1
automotive           1
Jr                   1
gridiron  player     1
aka                  1
Jules Engel          1
sports               1
of                   1
social               1
man                  1
of the Year          1
Name: info_1, dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have extracted all of the `known_for` information present in `info_1`.
- It is time to proceed with extracting the same from `info_2`, the column that should contain the bulk of this feature's values.

#### Extracting Category to `known_for_1` Column from `info_2`

In [38]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for_1'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!=''].sample(2)

There are 122730 values in extract_to column.
CPU times: total: 2min 24s
Wall time: 2min 24s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1,known_for_2,known_for_3
46962,21,Natasha Spender,", 91, British musician and writer, widow of Stephen Spender.",https://en.wikipedia.org/wiki/Natasha_Spender,7,2010,October,None,None,musician and,widow of Stephen Spender,None,None,None,None,None,None,None,None,91.0,None,United Kingdom of Great Britain and Northern Ireland,None,arts,,
129303,23,Antonio Fernández,", 79, Spanish football player and coach .",https://en.wikipedia.org/wiki/Antonio_Fern%C3%A1ndez_(footballer),3,2022,January,Málaga,None,player and coach,None,None,None,None,None,None,None,None,None,79.0,None,Spain,None,sports,,


<IPython.core.display.Javascript object>

#### Extracting Category to `known_for_2` Column from `info_2`

In [39]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for_2'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df['known_for_1']!= '') & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')
df[df[extract_to]!=''].sample(2)

There are 32445 values in extract_to column.
CPU times: total: 3min 32s
Wall time: 3min 32s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for_1,known_for_2,known_for_3
35811,3,William Brice,", 86, American painter and UCLA professor, injuries from fall.",https://en.wikipedia.org/wiki/William_Brice,1,2008,March,None,None,and UCLA,injuries from fall,None,None,None,None,None,None,None,None,86.0,None,United States of America,None,academia,arts,
97167,17,Bullumba Landestoy,", 93, Dominican pianist and composer.",https://en.wikipedia.org/wiki/Bullumba_Landestoy,5,2018,July,None,None,and,None,None,None,None,None,None,None,None,None,93.0,None,Dominican Republic,None,arts,arts,


<IPython.core.display.Javascript object>

#### Extracting Category to `known_for_3` Column from `info_2`

In [40]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for_3'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df['known_for_2']!= '') & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df[extract_to]!=""])} values in extract_to column.')

There are 2624 values in extract_to column.
CPU times: total: 1min 2s
Wall time: 1min 2s


<IPython.core.display.Javascript object>

In [41]:
print("dunzo!")
chime.success()

dunzo!


<IPython.core.display.Javascript object>